In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import loaddata, iblayer, trainutils

# Dictionary to 
cfg = {}
cfg['squaredIB'] = True
cfg['n_batch']   = 128
cfg['n_epochs']  = 150
cfg['n_noisevar_batch'] = 1000 
cfg['report_every']  = 10

betavals = 10**np.linspace(-5, 0.1, 30, endpoint=True)

savedirbase  = str(pathlib.Path().absolute()) + '/saveddata-v102/'
savedir = savedirbase + 'MNIST'

tfconfig = tf.ConfigProto()
tfconfig.gpu_options.allow_growth=True



In [ ]:
class Network(object):
    def __init__(self, input_dim, output_dim):
        self.input_dim      = input_dim
        self.output_dim     = output_dim
        
        
        self.iblayerobj     = iblayer.NoisyIBLayer(n_noisevar_batch=cfg['n_noisevar_batch'], 
                                                   init_noisevar=0.01, init_kdewidth=-20)
        
        self.true_outputs   = tf.placeholder(tf.float32, [None,output_dim,], name='true_outputs')

        # TODO: build the network
        self.layers = []
        self.layers.append( tf.placeholder(tf.float32, [None,input_dim,], name='X') ) # tf.keras.layers.Input([input_dim,])
        self.layers.append( tf.keras.layers.Dense(512, activation=tf.nn.relu, name='encoder_0')(self.layers[-1]) )
        self.layers.append( tf.keras.layers.Dense(512, activation=tf.nn.relu, name='encoder_1')(self.layers[-1]) )
        self.layers.append( tf.keras.layers.Dense(2  , activation=None, name='encoder_2')(self.layers[-1]) )
        self.layers.append( self.iblayerobj(self.layers[-1]) )
        self.layers.append( tf.keras.layers.Dense(512, activation=tf.nn.relu, name='decoder_0')(self.layers[-1]) )
        self.layers.append( tf.keras.layers.Dense(output_dim, activation=None, name='decoder_1')(self.layers[-1]) )

        self.inputs         = self.layers[0]
        self.predictions    = self.layers[-1]

        f                   = tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.true_outputs, logits=self.predictions)
        self.cross_entropy  = tf.reduce_mean(f) # cross entropy
        self.accuracy       = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(self.predictions, 1), tf.argmax(self.true_outputs, 1)), tf.float32))

In [ ]:
data           = loaddata.load_data('MNIST')
input_dim      = data['trn_X'].shape[1]
output_dim     = data['trn_Y'].shape[1]

In [ ]:
# Train the base model, without compression
tf.reset_default_graph()
with tf.Session(config=tfconfig) as sess:
    print("Making base model")
    n = Network(input_dim, output_dim)
    sess.run(tf.global_variables_initializer())
    trainutils.train(sess, mode='ce', beta=0.0, net=n, cfg=cfg, 
                data=data, savedir=savedir+'/basemodel')
    print("Model saved in path: %s" % savedir)
    del n


In [ ]:
for mode in ['nlIB', 'VIB']:
    for beta in betavals:
        if np.isclose(beta, 0):
            continue
        tf.reset_default_graph()
        with tf.Session(config=tfconfig) as sess:
            n = Network(input_dim, output_dim)
            sess.run(tf.global_variables_initializer())
            tf.train.Saver().restore(sess, tf.train.latest_checkpoint(savedir+'/basemodel'))

            sqmode = 'sq'  if cfg['squaredIB'] else 'reg'
            runndx = 0
            savename = savedir + '/results-%s-%0.5f-%s-run%d' % (mode, beta, sqmode, runndx)
            trainutils.train(sess, mode=mode, beta=beta, net=n, cfg=cfg, data=data, savedir=savename,
                  optimization_callback=n.iblayerobj.optimize_eta)
            print("Model saved in path: %s" % savedir)
            del n
            print()
            print()

In [ ]:
# plt.scatter(act[:,0], act[1,:], alpha=0.1)


In [ ]:
betavals